In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
import matplotlib.pyplot as plt

In [4]:
import seaborn as sns

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, accuracy_score, f1_score

In [7]:
df = pd.read_csv('C:/Users/anmis/Downloads/archive (1)/realest.csv')

In [8]:
df.head()

,Price,Bedroom,Space,Room,Lot,Tax,Bathroom,Garage,Condition
0,53.0,2.0,967.0,5.0,39.0,652.0,1.5,0.0,0.0
1,55.0,2.0,815.0,5.0,33.0,1000.0,1.0,2.0,1.0
2,56.0,3.0,900.0,5.0,35.0,897.0,1.5,1.0,0.0
3,58.0,3.0,1007.0,6.0,24.0,964.0,1.5,2.0,0.0
4,64.0,3.0,1100.0,7.0,50.0,1099.0,1.5,1.5,0.0


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 157 entries, 0 to 156
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Price      156 non-null    float64
 1   Bedroom    156 non-null    float64
 2   Space      146 non-null    float64
 3   Room       156 non-null    float64
 4   Lot        146 non-null    float64
 5   Tax        147 non-null    float64
 6   Bathroom   156 non-null    float64
 7   Garage     156 non-null    float64
 8   Condition  156 non-null    float64
dtypes: float64(9)
memory usage: 11.2 KB


In [10]:
df.describe()

,Price,Bedroom,Space,Room,Lot,Tax,Bathroom,Garage,Condition
count,156.000000,156.000000,146.000000,156.000000,146.000000,147.000000,156.000000,156.000000,156.000000
mean,56.474359,3.166667,1097.246575,6.500000,32.808219,911.707483,1.480769,0.846154,0.230769
std,12.875307,1.348037,462.540698,1.675247,8.457859,443.263430,0.529408,0.808454,0.422682
min,32.000000,1.000000,539.000000,4.000000,24.000000,418.000000,1.000000,0.000000,0.000000
25%,46.000000,2.000000,805.250000,5.000000,25.000000,652.500000,1.000000,0.000000,0.000000
50%,55.000000,3.000000,965.500000,6.000000,30.000000,821.000000,1.500000,1.000000,0.000000
75%,65.000000,4.000000,1220.500000,7.000000,37.000000,1012.500000,2.000000,1.500000,0.000000
max,90.000000,8.000000,2295.000000,12.000000,50.000000,2752.000000,3.000000,2.000000,1.000000


In [11]:
df.isnull().sum() 

Price         1
Bedroom       1
Space        11
Room          1
Lot          11
Tax          10
Bathroom      1
Garage        1
Condition     1
dtype: int64

In [12]:
df = df.dropna()

In [13]:
# Checking for missing values in each column
missing_values = df.isnull().sum()

In [14]:
print(missing_values[missing_values > 0])

Series([], dtype: int64)


In [17]:
# Filling missing values in numerical columns with the mean
numerical_cols = df.select_dtypes(include=['float64', 'int64']).columns

In [18]:
df[numerical_cols] = df[numerical_cols].fillna(df[numerical_cols].mean())

In [19]:
for col in numerical_cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    outliers = df[(df[col] < Q1 - 1.5 * IQR) | (df[col] > Q3 + 1.5 * IQR)]
    print(f"Outliers detected in {col}: {len(outliers)}")

Outliers detected in Price: 0
Outliers detected in Bedroom: 4
Outliers detected in Space: 17
Outliers detected in Room: 6
Outliers detected in Lot: 0
Outliers detected in Tax: 5
Outliers detected in Bathroom: 0
Outliers detected in Garage: 0
Outliers detected in Condition: 29


In [20]:
# Removing rows with outliers
for col in numerical_cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    df = df[~((df[col] < Q1 - 1.5 * IQR) | (df[col] > Q3 + 1.5 * IQR))]

# Build Classification Models

In [21]:
median_price = df['Price'].median()

In [22]:
df['Price_category'] = df['Price'].apply(lambda x: 1 if x >= median_price else 0)

In [23]:
X = df.drop(['Price', 'Price_category'], axis=1)

In [24]:
y = df['Price_category']

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Logistic Regression

In [26]:
from sklearn.linear_model import LogisticRegression

In [27]:
log_reg = LogisticRegression()

In [28]:
log_reg.fit(X_train, y_train)

C:\Users\anmis\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [29]:
y_pred_log = log_reg.predict(X_test)

In [30]:
# Evaluate Logistic Regression
print("Logistic Regression Accuracy:", accuracy_score(y_test, y_pred_log))

Logistic Regression Accuracy: 0.8666666666666667


In [31]:
print("Logistic Regression F1 Score:", f1_score(y_test, y_pred_log))

Logistic Regression F1 Score: 0.8333333333333334


In [32]:
from sklearn.ensemble import RandomForestClassifier

In [33]:
rf_clf = RandomForestClassifier(random_state=42)

In [34]:
rf_clf.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [35]:
y_pred_rf_clf = rf_clf.predict(X_test)

In [36]:
print("Random Forest Classifier F1 Score:", f1_score(y_test, y_pred_rf_clf))

Random Forest Classifier F1 Score: 0.9090909090909091


In [37]:
# Evaluate Random Forest Classifier
print("Random Forest Classifier Accuracy:", accuracy_score(y_test, y_pred_rf_clf))

Random Forest Classifier Accuracy: 0.9333333333333333
